In [108]:
import gpxpy
import pandas as pd
import os

module_path = os.path.dirname("__file__")
folder_path = os.path.join(module_path, "data_raw/")

def read_gpx(file: str) -> pd.DataFrame:
    df = None
    points = []
    with open(file) as f:
        gpx = gpxpy.parse(f)
    for segment in gpx.tracks[0].segments:
        for p in segment.points:
            points.append({
                'time': p.time,
                'latitude': p.latitude,
                'Longitude': p. longitude,
                'elevation': p.elevation
            })
    df = pd.DataFrame.from_records(points)
    return df

df1 = read_gpx(folder_path+'recovery.01-Mar-2022-1533.gpx')
df2 = read_gpx(folder_path+'recovery.05-Mar-2022.1025.gpx')
df3 = read_gpx(folder_path+'recovery.25-May-2022-0907.gpx')
df3 = df3.drop(index=0).reset_index(drop=True)
df3

,time,latitude,Longitude,elevation
0,2022-05-26 14:05:32.284000+00:00,6.268901,-75.593216,1548.51
1,2022-05-26 14:05:34+00:00,6.268870,-75.593087,1551.91
2,2022-05-26 14:05:36+00:00,6.268792,-75.593007,1551.53
3,2022-05-26 14:05:37+00:00,6.268722,-75.592900,1551.37
4,2022-05-26 14:05:38+00:00,6.268672,-75.592772,1551.04
...,...,...,...,...
652,2022-05-26 14:32:21+00:00,6.208590,-75.571338,1553.61
653,2022-05-26 14:32:27+00:00,6.208485,-75.571341,1548.82
654,2022-05-26 14:32:34+00:00,6.208457,-75.571435,1549.02
655,2022-05-26 14:32:41+00:00,6.208468,-75.571525,1559.75


In [109]:
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime, timedelta

def calculate_speed(df):
    lat, lon, time = df['latitude'], df['Longitude'], df['time']
    elev = df['elevation']
    distance = [0]
    speed = [0]
    time = [datetime.fromisoformat(str(t)) for t in time]
    
    # Convert latitudes and longitudes to km
    lat_km = lat.apply(lambda x: geodesic((x, lon[0]), (x, lon[0]+1)).km)
    lon_km = lon.apply(lambda x: geodesic((lat[0], x), (lat[0]+1, x)).km)

    for i in range(1, len(lat)):
        # Calculate the distance in km
        prev_point = (lat[i-1], lon[i-1])
        curr_point = (lat[i], lon[i])
        dist = geodesic(prev_point, curr_point).km
        distance.append(dist)

        # Calculate the time difference in hours
        prev_time = time[i-1]
        curr_time = time[i]
        time_diff = (curr_time - prev_time).total_seconds() / 3600

        # Calculate the speed in km/h
        try:
            curr_speed = dist / time_diff
        except:
            curr_speed = 0
        
        speed.append(curr_speed)

    # Add the calculated values to the dataframe
    df['lat_km'] = lat_km
    df['lon_km'] = lon_km
    df['time_hrs'] = [(t - time[0]).total_seconds() / 3600 for t in time]
    df['distance_km'] = distance
    df['speed_kph'] = speed
    
    df = df[df['speed_kph'] <= 100]
    return df


In [110]:
data1_generate =calculate_speed(df1)
data1_generate

,time,latitude,Longitude,elevation,lat_km,lon_km,time_hrs,distance_km,speed_kph
0,2022-03-01 20:33:48+00:00,6.297476,-75.578192,1668.879307,110.652205,110.58986,0.000000,0.000000,0.000000
1,2022-03-01 20:33:49+00:00,6.297476,-75.578146,1668.386143,110.652205,110.58986,0.000278,0.005092,18.330670
2,2022-03-01 20:33:57+00:00,6.297460,-75.578169,1667.904453,110.652208,110.58986,0.002500,0.003041,1.368269


In [111]:
data2_generate =calculate_speed(df2)
data2_generate.head(5)
data2_generate.shape

(18573, 9)

In [112]:
data3_generate =calculate_speed(df3)
data3_generate.head(5)

,time,latitude,Longitude,elevation,lat_km,lon_km,time_hrs,distance_km,speed_kph
0,2022-05-26 14:05:32.284000+00:00,6.268901,-75.593216,1548.51,110.658241,110.58973,0.000000,0.000000,0.000000
1,2022-05-26 14:05:34+00:00,6.268870,-75.593087,1551.91,110.658248,110.58973,0.000477,0.014781,31.009652
2,2022-05-26 14:05:36+00:00,6.268792,-75.593007,1551.53,110.658264,110.58973,0.001032,0.012350,22.230226
3,2022-05-26 14:05:37+00:00,6.268722,-75.592900,1551.37,110.658279,110.58973,0.001310,0.014100,50.760485
4,2022-05-26 14:05:38+00:00,6.268672,-75.592772,1551.04,110.658289,110.58973,0.001588,0.015142,54.510713


In [113]:
import plotly.graph_objects as go

def plot_velocity(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['time'], y=df['speed_kph'], mode='lines', name='Speed (km/h)'))
    fig.update_layout(title='Velocity over Time', xaxis_title='Time', yaxis_title='Velocity (km/h)')
    fig.show()


In [114]:
plot_velocity(data1_generate)

In [115]:
plot_velocity(data2_generate)

In [116]:
plot_velocity(data3_generate)